## 先来创建一个回答问题和调用工具的简单agent

读取.env中的环境配置文件

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from rich import print as rprint

# 获取项目根目录（当前文件所在目录的上一级）
project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
env_path = project_root / '.env'

# 加载 .env 文件
load_dotenv(dotenv_path=env_path)

required_vars = ['OPENAI_API_KEY', 'OPENAI_API_BASE', 'MODEL_NAME']
missing_vars = [var for var in required_vars if not os.environ.get(var)]

if missing_vars:
    rprint(f"[red]警告: 以下环境变量未在 .env 文件中配置: {', '.join(missing_vars)}[/red]")
else:
    rprint(f"[green]✓ 环境变量已从 {env_path} 成功加载[/green]")

✓ 环境变量已从 /home/ruoshui/Documents/MyProject/agent/langchain-academy-learning/.env 成功加载

**这部分接入好用的国产大模型**

In [ ]:
from langchain_openai import ChatOpenAI

# 从环境变量中读取模型名称
# OPENAI_API_KEY 和 OPENAI_API_BASE 也会自动从环境变量中读取
model_name = os.environ.get("MODEL_NAME")

llm = ChatOpenAI(model=model_name, temperature=0)

rprint(f"[cyan]当前使用模型: {model_name}[/cyan]")

# 注意：请在 .env 文件中配置模型名称
# 例如：
# MODEL_NAME=deepseek-ai/DeepSeek-V3.2-Exp

当前使用模型: deepseek-ai/DeepSeek-V3.2-Exp

这个agent主要是这么走的：
1. Agent 收到消息。
2. 思考：它分析语义，发现需要查天气，且只要调用 get_weather 工具就能解决。
3. 行动：它自动提取出 "sf" 作为参数，在后台偷偷运行了 get_weather("sf")。
4. 果：函数返回 "It's always sunny in sf!"。
5. 回答：Agent 把这个结果组织成自然语言，最终回复

In [8]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model= llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

rprint(response)

{
    'messages': [
        HumanMessage(
            content='what is the weather in sf',
            additional_kwargs={},
            response_metadata={},
            id='33d79436-a101-45d3-ba72-bc4f28c6bdd8'
        ),
        AIMessage(
            content="I'll check the weather in San Francisco for you.",
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 26,
                    'prompt_tokens': 161,
                    'total_tokens': 187,
                    'completion_tokens_details': {
                        'accepted_prediction_tokens': None,
                        'audio_tokens': None,
                        'reasoning_tokens': 0,
                        'rejected_prediction_tokens': None
                    },
                    'prompt_tokens_details': None
                },
                'model_provider': 'openai',
                'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp',
                'system_fingerprint': '',
                'id': '019aa58d54e5a9f9ef6708cca1e646f6',
                'finish_reason': 'tool_calls',
                'logprobs': None
            },
            id='lc_run--5a3e5c8e-7632-42b7-8512-81c6282d6496-0',
            tool_calls=[
                {
                    'name': 'get_weather',
                    'args': {'city': 'San Francisco'},
                    'id': '019aa58d63b6a34f9d961a3733f870e1',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 161,
                'output_tokens': 26,
                'total_tokens': 187,
                'input_token_details': {},
                'output_token_details': {'reasoning': 0}
            }
        ),
        ToolMessage(
            content="It's always sunny in San Francisco!",
            name='get_weather',
            id='37f061d4-0a26-4a87-be9b-b3cef7045d52',
            tool_call_id='019aa58d63b6a34f9d961a3733f870e1'
        ),
        AIMessage(
            content="The weather in San Francisco is sunny! According to the weather data, it's always sunny 
there.",
            additional_kwargs={'refusal': None},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 20,
                    'prompt_tokens': 199,
                    'total_tokens': 219,
                    'completion_tokens_details': {
                        'accepted_prediction_tokens': None,
                        'audio_tokens': None,
                        'reasoning_tokens': 0,
                        'rejected_prediction_tokens': None
                    },
                    'prompt_tokens_details': None
                },
                'model_provider': 'openai',
                'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp',
                'system_fingerprint': '',
                'id': '019aa58d64ce207b0fce3e833906cbfc',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='lc_run--b1cbbe68-a93c-4625-a295-4f07cc7ce038-0',
            usage_metadata={
                'input_tokens': 199,
                'output_tokens': 20,
                'total_tokens': 219,
                'input_token_details': {},
                'output_token_details': {'reasoning': 0}
            }
        )
    ]
}

## 现在来分析一下返回的message包含哪些部分：

```python
{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='b81d41c5-1b48-4fea-b87f-9ff140768610'),
  AIMessage(content="I'll check the weather in San Francisco for you.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 161, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'id': '019a9f3fc048d842155a2a303d8f3a51', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f2fbdca5-79af-46dc-99d6-1fd233e7dcf9-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '019a9f3fcf114d590004d96e71133cf4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 161, 'output_tokens': 26, 'total_tokens': 187, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='f8da9876-fab6-4e7e-8e34-46d00790eea5', tool_call_id='019a9f3fcf114d590004d96e71133cf4'),
  AIMessage(content="According to the weather information, it's always sunny in San Francisco!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 199, 'total_tokens': 213, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'deepseek-ai/DeepSeek-V3.2-Exp', 'system_fingerprint': '', 'id': '019a9f3fd05a409471a3d910a1eacdbe', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--7eeba0b5-615f-4555-8169-da469d0c5486-0', usage_metadata={'input_tokens': 199, 'output_tokens': 14, 'total_tokens': 213, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}})]}
```

返回的message是一个字典，展示了从“用户提问”到“最终回答”的完整**思考与执行链路**。它是一个列表，包含了 Agent 经历的 4 个步骤：

-----

### 用户提问 (`HumanMessage`)

```python
HumanMessage(content='what is the weather in sf', ...)
```
-----

### AI 的思考与决策 (`AIMessage`)

**这是最关键的一步**，包含了很多信息：

```python
AIMessage(
    content="I'll check the weather in San Francisco for you.", 
    tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, ...}], 
    finish_reason='tool_calls', 
    ...
)
```

  * **`content`**：AI 给出的中间回复（有时候这里是空的，取决于模型）。
  * **`tool_calls` **：
      * AI 分析了用户的话，决定它**不能直接回答**，而是需要调用工具。
      * **`name`: 'get\_weather'** -\> AI 选择调用`get_weather` 函数。
      * **`args`: {'city': 'San Francisco'}** -\> AI 自动把用户口语的 "sf" 转换成了规范的参数 "San Francisco"。
  * **`finish_reason`: 'tool\_calls'**：这告诉 LangChain 框架：“**且慢！我还没说完，请帮我运行一下工具，然后再把结果告诉我。**”

-----

### 工具执行 (`ToolMessage`)

```python
ToolMessage(
    content="It's always sunny in San Francisco!", 
    name='get_weather', 
    tool_call_id='019a9f3fcf114d590004d96e71133cf4'
)
```

  * **`content`**：函数 `get_weather("San Francisco")` 的返回值。
  * **`tool_call_id`**：这是一个“回执单号”。它必须和上一幕 AI 发出的 `id` 完全一致。这样 Agent 才知道这个结果对应的是哪一次请求。

-----

### 最终回答 (`AIMessage`)

```python
AIMessage(
    content="According to the weather information, it's always sunny in San Francisco!", 
    finish_reason='stop', 
    ...
)
```

  * **综合**：AI 拿到了第三幕的工具结果，结合第一幕的用户问题，生成了最终的自然语言回答。
  * **`finish_reason`: 'stop'**：AI 告诉框架：“**我任务完成了，可以把这句话展示给用户了。**”

-----

### 数据流向

1.  **输入：** `[用户问题]`
2.  **LLM 输出：** `[用户问题, AI想调工具]` -\> **暂停**
3.  **代码执行：** `[用户问题, AI想调工具, 工具结果]` -\> **喂回给 LLM**
4.  **LLM 最终输出：** `[用户问题, AI想调工具, 工具结果, 最终答案]`



## 更真实的一个代理


1. 详细的系统提示以获得更好的代理行为

In [9]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

2. 创建与外部数据集成的工具
   

In [10]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# 定义上下文结构
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str



AI的Schema
- 当 LangChain 把这个工具介绍给 AI 时，它会把 runtime 参数抹去。
- AI 看到的工具定义：get_user_location()
- AI 的理解：“有一个工具叫查询用户位置，我直接调用它就行，不需要我提供任何参数。”
- AI 的行为：当用户问“我在哪？”时，AI 只是简单地输出一个调用指令：Call tool: get_user_location（括号里是空的）。


代码执行的真相（Runtime）
当工具真正开始运行时，LangChain 框架会把后台准备好的 Context 偷偷塞给这个函数。

代码逻辑：
- AI 没传参数？没关系。
- 我看了一眼 runtime.context（后台偷偷塞进来的）。
- 哦，原来现在的 user_id 是 "1"。
- 那我返回 "Florida"。


In [11]:
# 上下文感知工具
# 在生成的工具 Schema 中，runtime 参数会被隐藏。
# AI 看到的只是一个名为 get_user_location 的工具，且不需要任何输入参数。
@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

3. 代理响应匹配特定模式，可以选择定义结构化响应格式
   LangChain 中非常核心且强大的功能：结构化输出（Structured Output）。之前我们让 AI 说话，它吐出来的是一段字符串（String）；现在我们用这段代码，要求 AI 吐出来的是一个对象（Object）。

   比如：
   ```python
    ResponseFormat(
        punny_response="Why did the sun go to school? To get brighter!",
        weather_conditions="Sunny"
    )
   ```

In [12]:
from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
# 这个ResponseFormat定义了希望 AI 最终返回的数据格式。会被 LangChain 翻译成 JSON Schema 发给 AI。



# punny_response: 这里其实实际上还是在写提示词
# weather_conditions: 任何有趣的天气信息（如果有的话）

# 如果 AI 刚才查了天气（调用了工具），它就会把天气填在这里。
# 如果用户只是问“讲个笑话”，AI 没查天气，它就会明智地把这里设为 None

@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

4. 添加一个记忆，允许代理记住之前的对话和上下文

In [13]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

    AI 拥有了一个持久化存储的能力，在这里是用内存进行存储


    注意： 这是为了方便演示才使用的`InemorySaver`。如果应用上线使用，需要换成 PostgresSaver 或 SqliteSaver，把记忆存到硬盘数据库里，这样服务器重启后用户的数据还在。

5. 进行组装并运行这个agent

In [14]:
from langchain_core.runnables import RunnableConfig # 导入类型


agent = create_agent(
    model= llm,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config: RunnableConfig = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)
rprint(response['structured_response'])
# print(response['structured_response'])


# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

rprint(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(
    punny_response="Well, it looks like Florida is living up to its sunny reputation! I'd say the weather is 
absolutely 'sun-sational' - it's so bright you might need to wear 'shades of glory'! Don't forget your sunscreen, 
or you might get 'burned' by how beautiful it is outside! 🌞",
    weather_conditions="It's always sunny in Florida!"
)

ResponseFormat(
    punny_response="You're 'weather' welcome! I'm always 'cloud' to help - it's my 'precipitation' in life! If you 
need another forecast, just 'rain' me a question anytime! ☔️",
    weather_conditions=None
)